In [2]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy

Using TensorFlow backend.


In [3]:
import os

In [4]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [21]:
import numpy as np
DATA_PATH = "/home/aaa/aatresh/Audio Processing/data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [22]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [23]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [24]:
save_data_to_array()

In [25]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=300, verbose=1, validation_data=(X_test, y_test_hot))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 5683 samples, validate on 3789 samples
Epoch 1/300
5683/5683 [==============================] - 1s 167us/step - loss: 7.7192 - acc: 0.3400 - val_loss: 1.2700 - val_acc: 0.4178
Epoch 2/300
5683/5683 [==============================] - 1s 128us/step - loss: 1.2896 - acc: 0.4408 - val_loss: 1.0104 - val_acc: 0.5838
Epoch 3/300
5683/5683 [==============================] - 1s 126us/step - loss: 1.0974 - acc: 0.5224 - val_loss: 0.9410 - val_acc: 0.6091
Epoch 4/300
5683/5683 [==============================] - 1s 119us/step - loss: 0.9763 - acc: 0.5747 - val_loss: 0.8374 - val_acc: 0.6791
Epoch 5/300
5683/5683 [==============================] - 1s 122us/step - loss: 0.9216 - acc: 0.6021 - val_loss: 0.7123 - val_acc: 0.7092
Epoch 6/300
5683/5683 [=========

5683/5683 [==============================] - 1s 110us/step - loss: 0.2282 - acc: 0.9184 - val_loss: 0.2614 - val_acc: 0.8997
Epoch 54/300
5683/5683 [==============================] - 1s 109us/step - loss: 0.2200 - acc: 0.9162 - val_loss: 0.2697 - val_acc: 0.9063
Epoch 55/300
5683/5683 [==============================] - 1s 111us/step - loss: 0.2235 - acc: 0.9173 - val_loss: 0.2981 - val_acc: 0.8926
Epoch 56/300
5683/5683 [==============================] - 1s 117us/step - loss: 0.2251 - acc: 0.9145 - val_loss: 0.3061 - val_acc: 0.8918
Epoch 57/300
5683/5683 [==============================] - 1s 129us/step - loss: 0.2051 - acc: 0.9212 - val_loss: 0.2803 - val_acc: 0.8984
Epoch 58/300
5683/5683 [==============================] - 1s 132us/step - loss: 0.1991 - acc: 0.9233 - val_loss: 0.2681 - val_acc: 0.9023
Epoch 59/300
5683/5683 [==============================] - 1s 125us/step - loss: 0.2089 - acc: 0.9222 - val_loss: 0.2797 - val_acc: 0.9013
Epoch 60/300
5683/5683 [=======================

5683/5683 [==============================] - 1s 134us/step - loss: 0.1290 - acc: 0.9518 - val_loss: 0.2843 - val_acc: 0.9076
Epoch 113/300
5683/5683 [==============================] - 1s 129us/step - loss: 0.1316 - acc: 0.9516 - val_loss: 0.2976 - val_acc: 0.8992
Epoch 114/300
5683/5683 [==============================] - 1s 130us/step - loss: 0.1191 - acc: 0.9539 - val_loss: 0.3140 - val_acc: 0.9060
Epoch 115/300
5683/5683 [==============================] - 1s 116us/step - loss: 0.1305 - acc: 0.9537 - val_loss: 0.3126 - val_acc: 0.9039
Epoch 116/300
5683/5683 [==============================] - 1s 110us/step - loss: 0.1213 - acc: 0.9546 - val_loss: 0.3077 - val_acc: 0.9000
Epoch 117/300
5683/5683 [==============================] - 1s 111us/step - loss: 0.1190 - acc: 0.9578 - val_loss: 0.3384 - val_acc: 0.9005
Epoch 118/300
5683/5683 [==============================] - 1s 111us/step - loss: 0.1193 - acc: 0.9555 - val_loss: 0.3005 - val_acc: 0.9042
Epoch 119/300
5683/5683 [================

5683/5683 [==============================] - 1s 117us/step - loss: 0.0830 - acc: 0.9708 - val_loss: 0.3221 - val_acc: 0.9100
Epoch 172/300
5683/5683 [==============================] - 1s 112us/step - loss: 0.0907 - acc: 0.9676 - val_loss: 0.3473 - val_acc: 0.9013
Epoch 173/300
5683/5683 [==============================] - 1s 131us/step - loss: 0.1003 - acc: 0.9639 - val_loss: 0.3293 - val_acc: 0.9047
Epoch 174/300
5683/5683 [==============================] - 1s 131us/step - loss: 0.0912 - acc: 0.9685 - val_loss: 0.3420 - val_acc: 0.9053
Epoch 175/300
5683/5683 [==============================] - 1s 131us/step - loss: 0.0952 - acc: 0.9676 - val_loss: 0.3142 - val_acc: 0.9034
Epoch 176/300
5683/5683 [==============================] - 1s 132us/step - loss: 0.1024 - acc: 0.9632 - val_loss: 0.3356 - val_acc: 0.9029
Epoch 177/300
5683/5683 [==============================] - 1s 124us/step - loss: 0.0978 - acc: 0.9674 - val_loss: 0.3279 - val_acc: 0.9076
Epoch 178/300
5683/5683 [================

5683/5683 [==============================] - 1s 116us/step - loss: 0.0791 - acc: 0.9717 - val_loss: 0.3428 - val_acc: 0.9047
Epoch 231/300
5683/5683 [==============================] - 1s 116us/step - loss: 0.0869 - acc: 0.9718 - val_loss: 0.3476 - val_acc: 0.9042
Epoch 232/300
5683/5683 [==============================] - 1s 114us/step - loss: 0.0715 - acc: 0.9764 - val_loss: 0.3428 - val_acc: 0.9039
Epoch 233/300
5683/5683 [==============================] - 1s 130us/step - loss: 0.0730 - acc: 0.9734 - val_loss: 0.3380 - val_acc: 0.9053
Epoch 234/300
5683/5683 [==============================] - 1s 129us/step - loss: 0.0742 - acc: 0.9750 - val_loss: 0.3680 - val_acc: 0.9034
Epoch 235/300
5683/5683 [==============================] - 1s 131us/step - loss: 0.0766 - acc: 0.9741 - val_loss: 0.3836 - val_acc: 0.9023
Epoch 236/300
5683/5683 [==============================] - 1s 131us/step - loss: 0.0848 - acc: 0.9725 - val_loss: 0.3408 - val_acc: 0.9039
Epoch 237/300
5683/5683 [================

5683/5683 [==============================] - 1s 117us/step - loss: 0.0632 - acc: 0.9762 - val_loss: 0.3589 - val_acc: 0.9087
Epoch 290/300
5683/5683 [==============================] - 1s 122us/step - loss: 0.0519 - acc: 0.9819 - val_loss: 0.3952 - val_acc: 0.9060
Epoch 291/300
5683/5683 [==============================] - 1s 157us/step - loss: 0.0637 - acc: 0.9778 - val_loss: 0.3528 - val_acc: 0.9092
Epoch 292/300
5683/5683 [==============================] - 1s 148us/step - loss: 0.0671 - acc: 0.9773 - val_loss: 0.3642 - val_acc: 0.9060
Epoch 293/300
5683/5683 [==============================] - 1s 149us/step - loss: 0.0619 - acc: 0.9787 - val_loss: 0.3825 - val_acc: 0.9079
Epoch 294/300
5683/5683 [==============================] - 1s 152us/step - loss: 0.0650 - acc: 0.9789 - val_loss: 0.3642 - val_acc: 0.9026
Epoch 295/300
5683/5683 [==============================] - 1s 148us/step - loss: 0.0595 - acc: 0.9792 - val_loss: 0.3828 - val_acc: 0.9034
Epoch 296/300
5683/5683 [================

In [27]:
sample = wav2mfcc('/home/aaa/aatresh/Audio Processing/data/go/0a9f9af7_nohash_0.wav')
sample_reshaped = sample.reshape(1,20,11,1)
print(get_labels()[0][np.argmax(model.predict(sample_reshaped))])

go


In [22]:
from pydub import AudioSegment

/usr/local/lib/python3.6/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [28]:
import pyaudio
import wave
import time
import sys
import pygame

ModuleNotFoundError: No module named 'pyaudio'

In [51]:
import socket
import numpy as np
from pygame import mixer
import time


try:
        mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print ("Socket created")

except socket.error:
        print ("Socket connection failed")

port = 8888
try:
        mySock.connect(("192.168.43.195", port))
        print ("Connected to RPi!")
except socket.error:
        print ("Not able to connect")



"""
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "out.wav"
while True:
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("* done recording")

        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        recording = AudioSegment.from_wav("./out.wav")
        recording+=20
        recording.export("./out.wav", format="wav")

        print("Before the sleep statement")
        mixer.init(frequency=44100)
        mixer.music.load("./out.wav")
        mixer.music.play()
        time.sleep(5)
        print("After the sleep statement")
        
"""

filenames=os.listdir("/home/aaa/aatresh/Audio Processing/New folder/")

while(1):
 


    for f in filenames:
        sample = wav2mfcc("/home/aaa/aatresh/Audio Processing/New folder/"+f)
        sample_reshaped = sample.reshape(1,20,11,1)
        message = get_labels("/home/aaa/aatresh/Audio Processing/data/")[0][np.argmax(model.predict(sample_reshaped))]   

        mixer.init(frequency=44100)
        mixer.music.load("/home/aaa/aatresh/Audio Processing/New folder/"+f)
        mixer.music.play()
        
        print(message)
        mySock.send(message.encode())
        data = mySock.recv(1024).decode()
        print ("Received from server: " + data)
        time.sleep(2)
    #    message = input("->")
        

#message="go"
"""
while(1):
    mySock.send(message.encode())
    data = mySock.recv(1024).decode()
    print ("Received from server: " + data)
"""

mySock.close()


KeyboardInterrupt: 